In [11]:
!pip install -q \
    sentence-transformers \
    faiss-cpu \
    lightfm \
    transformers \
    scikit-learn \
    pandas \
    numpy


In [12]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json
from lightfm import LightFM
from lightfm.data import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline



# ===============================
# 1. Load and Encode Keywords
# ===============================

# Load SBERT model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Load training data (make sure the path is correct)
with open('/content/edinburgh-keywords_train.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)

# Extract keywords
keywords = list(train_data['np2count'].keys())

# Encode all keywords into dense vectors
keyword_embeddings = model.encode(keywords, show_progress_bar=True)

# Normalize embeddings for cosine similarity
keyword_embeddings = keyword_embeddings / np.linalg.norm(keyword_embeddings, axis=1, keepdims=True)

# ===============================
# 2. Create FAISS Index
# ===============================

# Get embedding dimension
d = keyword_embeddings.shape[1]

# Use IndexFlatIP for inner product (cosine similarity after normalization)
index = faiss.IndexFlatIP(d)

# Add embeddings to index
index.add(keyword_embeddings)

# ===============================
# 3. Function to Find Similar Keywords
# ===============================

def find_similar_keywords(query_keywords, top_n=1):
    # Encode and normalize query keywords
    query_embeddings = model.encode(query_keywords)
    query_embeddings = query_embeddings / np.linalg.norm(query_embeddings, axis=1, keepdims=True)

    # Search in FAISS index
    D, I = index.search(query_embeddings, top_n)

    # Map back to actual keyword strings
    return [[keywords[i] for i in row] for row in I]

Batches:   0%|          | 0/307 [00:00<?, ?it/s]

In [13]:
# ===============================
# 1. Load Data
# ===============================

with open('/content/edinburgh-keywords_train.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)

# ===============================
# 2. Extract Users and Restaurants
# ===============================

# Collect all unique users and restaurants
all_users = set()
all_restaurants = set()

for kw, users in train_data['np2users'].items():
    all_users.update(users)

for kw, rests in train_data['np2rests'].items():
    all_restaurants.update(rests.keys())

# ===============================
# 3. Build LightFM Dataset
# ===============================

dataset = Dataset()
dataset.fit(users=all_users, items=all_restaurants)

# Fit additional users/items dynamically
for kw in train_data['np2users']:
    users = train_data['np2users'][kw]
    items = train_data['np2rests'].get(kw, {}).keys()
    dataset.fit_partial(users=users, items=items)

# ===============================
# 4. Build Interactions Matrix
# ===============================

(interactions, _) = dataset.build_interactions(
    ( (user, restaurant)
      for kw in train_data['np2rests']
      for restaurant in train_data['np2rests'][kw]
      for user in train_data['np2users'].get(kw, []) )
)

# ===============================
# 5. Train LightFM Model
# ===============================

model_cf = LightFM(loss='warp')
model_cf.fit(interactions, epochs=10, num_threads=2)

In [14]:
# ===============================
# Hybrid Recommender Function
# ===============================

def recommend_restaurants(user_id, user_keywords, top_k=10):
    """
    Recommend restaurants based on:
    1. Keyword similarity (using FAISS)
    2. Keyword-to-restaurant mapping
    3. LightFM collaborative filtering scores

    Params:
        user_id: str - User identifier
        user_keywords: list of str - Keywords expressing user preferences
        top_k: int - Number of restaurants to recommend

    Returns:
        List of top-k recommended restaurant names
    """

    # Step 1: Map user keywords to closest known ones (using FAISS)
    mapped_keywords = [
        find_similar_keywords([kw])[0][0]
        for kw in user_keywords
        if kw
    ]

    # Step 2: Score restaurants based on keyword co-occurrence
    scores = np.zeros(len(restaurant_set))

    for kw in mapped_keywords:
        for res in train_data['np2rests'].get(kw, {}):
            res_idx = restaurant_set.index(res)
            scores[res_idx] += 1
    # Step 3: Add LightFM collaborative filtering scores
    user_map = dataset.mapping()[0]  # {user_id_str: internal_id}
    item_map = dataset.mapping()[2]  # {restaurant_name: internal_id}
    item_reverse_map = {v: k for k, v in item_map.items()}

    known_user = user_map.get(user_id)

    if known_user is not None:
        # Predict CF scores for all restaurants
        cf_scores = model_cf.predict(known_user, np.arange(len(item_map)))

        # Align CF scores with restaurant_set indices
        for rest_name, internal_idx in item_map.items():
            external_idx = restaurant_set.index(rest_name)
            scores[external_idx] += cf_scores[internal_idx]

    # Step 4: Get top-k restaurant indices
    top_indices = np.argsort(scores)[::-1][:top_k]

    # Step 5: Return restaurant names
    return [restaurant_set[i] for i in top_indices]


In [15]:
model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

def re_rank_with_llm(user_keywords, candidates, top_k=None):
    prompt = (
        f"User interests: {', '.join(user_keywords)}.\n"
        f"Candidate restaurants: {', '.join(candidates)}.\n"
        "Rank the restaurants in order of best fit for the user's interests. "
        "Output a comma-separated list of restaurant names only."
    )

    response = generator(prompt, max_new_tokens=100, do_sample=False)[0]['generated_text']
    ranked = [r.strip() for r in response.split(',') if r.strip()]
    valid_ranked = [r for r in ranked if r in candidates]

    return valid_ranked[:top_k] if top_k else valid_ranked or candidates


Device set to use cpu


In [16]:
def precision_at_k(predicted, actual, k=10):
    """
    Compute Precision@K

    Args:
        predicted (list): List of recommended items
        actual (list): List of relevant (ground truth) items
        k (int): Top-K cutoff

    Returns:
        float: Precision@K score
    """
    if not predicted:
        return 0.0

    predicted = predicted[:k]
    hits = set(predicted) & set(actual)
    return len(hits) / k


def recall_at_k(predicted, actual, k=10):
    """
    Compute Recall@K

    Args:
        predicted (list): List of recommended items
        actual (list): List of relevant (ground truth) items
        k (int): Top-K cutoff

    Returns:
        float: Recall@K score
    """
    if not actual:
        return 0.0

    predicted = predicted[:k]
    hits = set(predicted) & set(actual)
    return len(hits) / len(actual)
